In [13]:
import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F

from torch.utils.data import DataLoader, Dataset, random_split, TensorDataset
from torchvision.transforms import Compose, ToTensor, Normalize, Resize, ToPILImage, CenterCrop, RandomResizedCrop
from torchvision.datasets import ImageFolder
from torchvision.models import alexnet, resnet18, inception_v3

from torchvision.models.alexnet import AlexNet_Weights
from torchvision.models.inception import Inception_V3_Weights
from torchvision.models.resnet import ResNet18_Weights

from download_rps import download_rps

In [17]:
data_path = "../data/"
train_data_path = data_path + "rps"
val_data_path = data_path + "rps-test-set"
download_rps(data_path)

rps folder already exists!
rps-test-set folder already exists!


In [19]:
# ImageNet statistics
# normalizer = Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
# composer = Compose([Resize(256), CenterCrop(224), ToTensor(), normalizer])

transformer = ResNet18_Weights.DEFAULT.transforms

train_data = ImageFolder(root=train_data_path, transform=transformer)
val_data = ImageFolder(root=val_data_path, transform=transformer)

# Builds a loader of each set
train_loader = DataLoader(train_data, batch_size=16, shuffle=True)
val_loader = DataLoader(val_data, batch_size=16)

In [11]:
resnet = resnet18(weights=ResNet18_Weights.DEFAULT)

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to C:\Users\Balint/.cache\torch\hub\checkpoints\resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 51.3MB/s]


In [12]:
def freeze_model(model):
    for parameter in model.parameters():
        parameter.requires_grad = False

In [13]:
freeze_model(resnet)

In [ ]:
torch.manual_seed(42)

